In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LightSource

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# download MNIST training and testing datasets, then prepare corresponding dataloaders (batch size = 100)
mnist_train = datasets.MNIST("../data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("../data", train=False, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(mnist_train, batch_size = 100, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size = 100, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# initialize the CNN architecture with 4 convolutional layers and 2 MLP layers for standard training
torch.manual_seed(0)

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

model_cnn = nn.Sequential(nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(),
                          nn.Conv2d(32, 32, 3, padding=1, stride=2), nn.ReLU(),
                          nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
                          nn.Conv2d(64, 64, 3, padding=1, stride=2), nn.ReLU(),
                          Flatten(),
                          nn.Linear(7*7*64, 100), nn.ReLU(),
                          nn.Linear(100, 10)).to(device)

In [4]:
#### Your task: complete the following function
def pgd(model, X, y, epsilon=0.1, alpha=0.02, num_iter=10, randomize=False):
    """ Construct PGD adversarial examples for the example (X,y)"""
    if randomize:
        delta = torch.rand_like(X, requires_grad=True)
        delta.data = delta.data * 2 * epsilon - epsilon
    else:
        delta = torch.zeros_like(X, requires_grad=True)
    for t in range(num_iter):
        loss = nn.CrossEntropyLoss()(model(X + delta), y)
        loss.backward()
        delta.data = (delta + alpha*delta.grad.detach().sign()).clamp(-epsilon,epsilon)
        delta.grad.zero_()
    return X + delta.detach()

In [5]:
#### Your task: complete the following functions
def epoch(loader, model, opt=None):
    """Standard training/evaluation epoch over the dataset"""
    total_loss, total_err = 0.,0.
    for X,y in loader:
        X,y = X.to(device), y.to(device)
        yp = model(X)
        loss = nn.CrossEntropyLoss()(yp,y)
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        total_err += (yp.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item()*X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

def epoch_adv(loader, model, attack, opt=None, **kwargs):
    """Adversarial training/evaluation epoch over the dataset"""
    total_loss, total_err = 0.,0.
    for X,y in loader:
        X,y = X.to(device), y.to(device)
        delta = attack(model, X, y, **kwargs)
        yp = model(X+delta)
        loss = nn.CrossEntropyLoss()(yp,y)
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        total_err += (yp.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item()*X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

In [8]:
# specify the optimizer as SGD
opt = optim.SGD(model_cnn.parameters(), lr=1e-1)

# standard training
for t in range(5):
    train_err, train_loss = epoch(train_loader, model_cnn, opt)
    test_err, test_loss = epoch(test_loader, model_cnn)
    adv_err, adv_loss = epoch_adv(test_loader, model_cnn, pgd)

    print(*("{:.6f}".format(i) for i in (train_err, test_err, adv_err)), sep="\t")

# save the standard trained model for further evaluation
torch.save(model_cnn.state_dict(), "model_cnn.pt")

0.077267	0.024500	0.167800
0.023183	0.020100	0.165800
0.015283	0.015900	0.170900
0.011700	0.013000	0.150100
0.009383	0.014600	0.179300


In [6]:
# use the same CNN architecture for robust training
model_cnn_robust = nn.Sequential(nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(),
                                 nn.Conv2d(32, 32, 3, padding=1, stride=2), nn.ReLU(),
                                 nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
                                 nn.Conv2d(64, 64, 3, padding=1, stride=2), nn.ReLU(),
                                 Flatten(),
                                 nn.Linear(7*7*64, 100), nn.ReLU(),
                                 nn.Linear(100, 10)).to(device)

In [10]:
# specify the optimizer as SGD
opt = optim.SGD(model_cnn_robust.parameters(), lr=1e-1)

# PGD-based adversarial training
for t in range(5):
    train_err, train_loss = epoch_adv(train_loader, model_cnn_robust, pgd, opt)
    test_err, test_loss = epoch(test_loader, model_cnn_robust)
    adv_err, adv_loss = epoch_adv(test_loader, model_cnn_robust, pgd)

    print(*("{:.6f}".format(i) for i in (train_err, test_err, adv_err)), sep="\t")

# save the standard trained model for further evaluation
torch.save(model_cnn_robust.state_dict(), "model_cnn_robust.pt")

0.219083	0.022200	0.044700


In [7]:
# load the standard trained and adversarially trained models
model_cnn.load_state_dict(torch.load("model_cnn.pt"))
model_cnn_robust.load_state_dict(torch.load("model_cnn_robust.pt"))

<All keys matched successfully>

In [8]:
def fgsm(model, X, y, epsilon=0.1):
    """ Construct FGSM adversarial examples for the example (X,y)"""
    delta = torch.zeros_like(X, requires_grad=True)
    loss = nn.CrossEntropyLoss()(model(X + delta), y)
    loss.backward()
    return epsilon * delta.grad.detach().sign()

In [10]:
# clean performance (no attack)
print("clean:", "{:.4f}".format(epoch(test_loader, model_cnn)[0]),
      "{:.4f}".format(epoch(test_loader, model_cnn_robust)[0]))

# evaluate both models using FGSM attack
print("FGSM: ", "{:.4f}".format(epoch_adv(test_loader, model_cnn, fgsm)[0]),
      "{:.4f}".format(epoch_adv(test_loader, model_cnn_robust, fgsm)[0]))

# evaluate both models using PGD attack
print("PGD (10 iter):", "{:.4f}".format(epoch_adv(test_loader, model_cnn, pgd, num_iter=10)[0]),
      "{:.4f}".format(epoch_adv(test_loader, model_cnn_robust, pgd, num_iter=10)[0]))

clean: 0.0128 0.0096
FGSM:  0.5438 0.0490
PGD (10 iter): 0.1737 0.0225


In [18]:
#### Your task (bonus): develop an attack method to achieve an attack success rate as high as possible. You can modify the following function if needed.

# You can try out some of the attack methods introduced in Lectures 3-4 or develop your unique creative attack.
# In principle, the performance of your attack should be better than FGSM or PGD, 10 iter;
# The higher attack success rates you can achieve, the higher credits you may receive.

def my_attack(model, X, y, epsilon=0.1):
  """ Construct adversarial examples for the example (X,y)"""
  delta = torch.zeros_like(X, requires_grad=True)
  alpha = epsilon/5 # initial step size was epsilon/10 and gave 0.1656 0.0223
  for _ in range(10):
    loss = nn.CrossEntropyLoss()(model(X + delta), y)
    loss.backward()
    delta.data += alpha * delta.grad.detach().sign()
    delta.data = torch.clamp(delta.data, -epsilon, epsilon)
    delta.grad.zero_()
  return X + delta.detach()

In [19]:
print("My Attack: ", "{:.4f}".format(epoch_adv(test_loader, model_cnn, my_attack)[0]), 
      "{:.4f}".format(epoch_adv(test_loader, model_cnn_robust, my_attack)[0]))

My Attack:  0.1737 0.0225


In [28]:
# Tried a version of zoo attack
# Changed epsilon to 0.2 and alpha to 0.05
def my_attack(model, X, y, epsilon=0.2, alpha=0.05, num_iterations=10):
    """ Construct adversarial examples on the examples X"""
    delta = torch.zeros_like(X, requires_grad=False)
    for i in range(num_iterations):
        for j in range(X.shape[1]):
            delta_j = torch.zeros_like(X)
            delta_j[:, j, :, :] = alpha
            loss_plus = nn.CrossEntropyLoss()(model(X + delta + delta_j), y)
            loss_minus = nn.CrossEntropyLoss()(model(X + delta - delta_j), y)
            grad_approx = (loss_plus - loss_minus) / (2 * alpha)
            delta[:, j, :, :] += alpha * torch.sign(grad_approx)
        delta = torch.clamp(delta, -epsilon, epsilon)
    return delta


print("My Attack: ", "{:.4f}".format(epoch_adv(test_loader, model_cnn, my_attack)[0]), 
      "{:.4f}".format(epoch_adv(test_loader, model_cnn_robust, my_attack)[0]))

My Attack:  0.0186 0.0117


In [30]:
# Changed epsilon to 0.2 and alpha to 0.1
def my_attack(model, X, y, epsilon=0.2, alpha=0.1, num_iterations=10):
    """ Construct adversarial examples on the examples X"""
    delta = torch.zeros_like(X, requires_grad=False)
    for i in range(num_iterations):
        for j in range(X.shape[1]):
            delta_j = torch.zeros_like(X)
            delta_j[:, j, :, :] = alpha
            loss_plus = nn.CrossEntropyLoss()(model(X + delta + delta_j), y)
            loss_minus = nn.CrossEntropyLoss()(model(X + delta - delta_j), y)
            grad_approx = (loss_plus - loss_minus) / (2 * alpha)
            delta[:, j, :, :] += alpha * torch.sign(grad_approx)
        delta = torch.clamp(delta, -epsilon, epsilon)
    return delta


print("My Attack: ", "{:.4f}".format(epoch_adv(test_loader, model_cnn, my_attack)[0]), 
      "{:.4f}".format(epoch_adv(test_loader, model_cnn_robust, my_attack)[0]))

My Attack:  0.0185 0.0117
